In [1]:
import numpy as np
from keras.applications import MobileNet
from keras.applications.mobilenet import preprocess_input
import keras
from keras import backend as K
from keras.layers.core import Dense,Dropout
from keras.layers import GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt

In [3]:
train_path='base_dir/train_dir'
valid_path='base_dir/val_dir'

num_train_samples=32247
num_val_samples=3511
train_batch_size=10
val_batch_size=10
image_size=224

In [4]:
train_steps=np.ceil(num_train_samples/train_batch_size)
val_steps=np.ceil(num_val_samples/val_batch_size)

In [5]:
print(train_steps)
print(val_steps)

3225.0
352.0


In [6]:
training_batches=ImageDataGenerator(
    preprocessing_function=\
    keras.applications.mobilenet.preprocess_input).flow_from_directory(
        train_path,
        target_size=(image_size,image_size),
        batch_size=train_batch_size
    )

Found 32247 images belonging to 5 classes.


In [7]:
validation_batches=ImageDataGenerator(
    preprocessing_function=\
    keras.applications.mobilenet.preprocess_input).flow_from_directory(
        valid_path,
        target_size=(image_size,image_size),
        batch_size=val_batch_size
    )

test_batches=ImageDataGenerator(
    preprocessing_function=\
    keras.applications.mobilenet.preprocess_input).flow_from_directory(
        valid_path,
        target_size=(image_size,image_size),
        batch_size=val_batch_size,
        shuffle=False
    )

Found 3511 images belonging to 5 classes.
Found 3511 images belonging to 5 classes.


In [8]:
mobile=keras.applications.mobilenet.MobileNet()

In [9]:
mobile.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 32)     128       
 ation)                                         

                                                                 
 conv_pw_8_bn (BatchNormaliz  (None, 14, 14, 512)      2048      
 ation)                                                          
                                                                 
 conv_pw_8_relu (ReLU)       (None, 14, 14, 512)       0         
                                                                 
 conv_dw_9 (DepthwiseConv2D)  (None, 14, 14, 512)      4608      
                                                                 
 conv_dw_9_bn (BatchNormaliz  (None, 14, 14, 512)      2048      
 ation)                                                          
                                                                 
 conv_dw_9_relu (ReLU)       (None, 14, 14, 512)       0         
                                                                 
 conv_pw_9 (Conv2D)          (None, 14, 14, 512)       262144    
                                                                 
 conv_pw_9

In [10]:
#for creating our custom model using mobilenet we will remmove last five layers from the model
# as they are used for a different i.e. base model for mobilenet
x=mobile.layers[-6].output
x=Dropout(0.25)(x)
x=GlobalAveragePooling2D()(x)
predictions=Dense(5,activation='softmax')(x)

model=Model(inputs=mobile.input,outputs=predictions)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 32)     128       
 ation)                                                      

                                                                 
 conv_pw_8_bn (BatchNormaliz  (None, 14, 14, 512)      2048      
 ation)                                                          
                                                                 
 conv_pw_8_relu (ReLU)       (None, 14, 14, 512)       0         
                                                                 
 conv_dw_9 (DepthwiseConv2D)  (None, 14, 14, 512)      4608      
                                                                 
 conv_dw_9_bn (BatchNormaliz  (None, 14, 14, 512)      2048      
 ation)                                                          
                                                                 
 conv_dw_9_relu (ReLU)       (None, 14, 14, 512)       0         
                                                                 
 conv_pw_9 (Conv2D)          (None, 14, 14, 512)       262144    
                                                                 
 conv_pw_9

In [11]:
for layer in model.layers[:-23]:
    layer.trainable=False


In [12]:
from keras.metrics import categorical_accuracy,top_k_categorical_accuracy

In [13]:
def top_3_accuracy(y_true,y_pred):
    return top_k_categorical_accuracy(y_true,y_pred,k=3)
def top_2_accuracy(y_true,y_pred):
    return top_k_categorical_accuracy(y_true,y_pred,k=2)

In [14]:
model.compile(Adam(learning_rate=0.01),loss='categorical_crossentropy',metrics=[categorical_accuracy,top_2_accuracy,top_3_accuracy])

In [15]:
#we are defining our own weights for the model and not using imagenet weights
#to do that we divide each class's number of images with the highest number of images present
# Ex: 23222/23222 and 23222/2194 and so on
class_weights={
    0: 1.0,
    1: 10.0,
    2: 4.0,
    3: 22.0,
    4: 22.0,
}

In [16]:
filepath="model.h5"

In [17]:
checkpoint=ModelCheckpoint(filepath,monitor='val_top_3_accuracy',
                           verbose=1,save_best_only=True,mode='max',
                           min_lr=0.00001)
reduce_lr=ReduceLROnPlateau(monitor='val_top_3_accuracy',
                            factor=0.5,patience=2,verbose=1,
                            mode='max',min_lr=0.00001)
callbacks_list=[checkpoint,reduce_lr]

In [ ]:
history=model.fit_generator(training_batches,
                           steps_per_epoch=train_steps,
                           class_weight=class_weights,
                           validation_data=validation_batches,
                           validation_steps=val_steps,
                           epochs=30,
                           verbose=1,
                           callbacks=callbacks_list)

C:\Users\akash\AppData\Local\Temp\ipykernel_4024\2376756131.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history=model.fit_generator(training_batches,


Epoch 1/30
3225/3225 [==============================] - ETA: 0s - loss: 4.9364 - categorical_accuracy: 0.3514 - top_2_accuracy: 0.6864 - top_3_accuracy: 0.8623
Epoch 1: val_top_3_accuracy improved from -inf to 0.95956, saving model to model.h5
3225/3225 [==============================] - 1387s 429ms/step - loss: 4.9364 - categorical_accuracy: 0.3514 - top_2_accuracy: 0.6864 - top_3_accuracy: 0.8623 - val_loss: 1.2637 - val_categorical_accuracy: 0.1467 - val_top_2_accuracy: 0.8189 - val_top_3_accuracy: 0.9596 - lr: 0.0100
Epoch 2/30
3225/3225 [==============================] - ETA: 0s - loss: 4.3495 - categorical_accuracy: 0.3852 - top_2_accuracy: 0.7246 - top_3_accuracy: 0.8910
Epoch 2: val_top_3_accuracy improved from 0.95956 to 0.96696, saving model to model.h5
3225/3225 [==============================] - 1342s 416ms/step - loss: 4.3495 - categorical_accuracy: 0.3852 - top_2_accuracy: 0.7246 - top_3_accuracy: 0.8910 - val_loss: 0.8146 - val_categorical_accuracy: 0.7405 - val_top_2_ac

3225/3225 [==============================] - ETA: 0s - loss: 2.2088 - categorical_accuracy: 0.5420 - top_2_accuracy: 0.8790 - top_3_accuracy: 0.9882
Epoch 16: val_top_3_accuracy did not improve from 0.96696

Epoch 16: ReduceLROnPlateau reducing learning rate to 7.812499825377017e-05.
3225/3225 [==============================] - 806s 250ms/step - loss: 2.2088 - categorical_accuracy: 0.5420 - top_2_accuracy: 0.8790 - top_3_accuracy: 0.9882 - val_loss: 1.1929 - val_categorical_accuracy: 0.4361 - val_top_2_accuracy: 0.8149 - val_top_3_accuracy: 0.9638 - lr: 1.5625e-04
Epoch 17/30
3225/3225 [==============================] - ETA: 0s - loss: 2.1617 - categorical_accuracy: 0.5472 - top_2_accuracy: 0.8793 - top_3_accuracy: 0.9895
Epoch 17: val_top_3_accuracy improved from 0.96696 to 0.96753, saving model to model.h5
3225/3225 [==============================] - 812s 252ms/step - loss: 2.1617 - categorical_accuracy: 0.5472 - top_2_accuracy: 0.8793 - top_3_accuracy: 0.9895 - val_loss: 1.1404 - va

In [ ]:
val_loss,val_cat_acc,val_top_2_acc,val_top_3_acc=\
model.evaluate_generator(test_batches,steps=val_steps)

print('val_loss: ',val_loss)
print('val_cat_acc: ',val_cat_acc)
print('val_top_2_acc: ',val_top_2_acc)
print('val_top_3_acc: ',val_top_3_acc)

In [ ]:
model.load_weights('model.h5')

In [ ]:
#checking again after loading our weights
val_loss,val_cat_acc,val_top_2_acc,val_top_3_acc=\
model.evaluate_generator(test_batches,steps=val_steps)

print('val_loss: ',val_loss)
print('val_cat_acc: ',val_cat_acc)
print('val_top_2_acc: ',val_top_2_acc)
print('val_top_3_acc: ',val_top_3_acc)

In [ ]:
predictions=model.predict_generator(test_batches,steps=val_steps,verbose=1)

In [ ]:
from sklearn.metrics import confusion_matrix
import itertools
test_labels=test_batches.classes

In [ ]:
def plot(cm,classes,normalize=False,title='Confusion Matrix',cmap=plt.cm.Blues):
    if normalize:
        cm=cm.astype('float')/cm.sum(axis=1)[:,np.newaxis]
        print('Normalized Confusion Matrix')
    else:
        print('CM without normalization')
    print(cm)
    
    plt.imshow(cm,interpolation='nearest',cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks=np.arange(len(classes))
    plt.xticks(tick_marks,classes,rotation=45)
    plt.yticks(tick_marks,classes)
    
    fmt='.2f' if normalize else 'd'
    thresh=cm.max()/2.
    for i,j in itertools.product(range(cm.shape[0]),range(cm.shape[1])):
        plt.text(j,i,format(cm[i,j],fmt),
                horizontalalignment="center",
                color="white" if cm[i,j]>thresh else "black")
    
    plt.xlabel('True label')
    plt.ylabel('Predicted label')
    plt.tight_layout()

cm=confusion_matrix(test_labels,predictions.argmax(axis=1))
cm_plot_labels=['No_DR', 'Mild', 'Moderate', 'Severe', 'Proliferate_DR']
plot(cm,cm_plot_labels)

## Deployment

In [74]:

%%writefile app.py
import tensorflow as tf
import streamlit as st


@st.cache(allow_output_mutation=True)
def load_model():
  model=tf.keras.models.load_model('model.h5',compile=False)
  return model
with st.spinner('Model is being loaded..'):
  model=load_model()

st.write("""
         # Diabetic Retinopathy Classification
         """
         )

file = st.file_uploader("Please upload an brain scan file", type=["jpg", "png","jpeg"])
import cv2
from PIL import Image, ImageOps
import numpy as np
st.set_option('deprecation.showfileUploaderEncoding', False)
def import_and_predict(image_data, model):
    
        size = (224,224)    
        image = ImageOps.fit(image_data, size, Image.ANTIALIAS)
        image = np.asarray(image)
        img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        #img_resize = (cv2.resize(img, dsize=(75, 75),    interpolation=cv2.INTER_CUBIC))/255.
        
        img_reshape = img[np.newaxis,...]
    
        prediction = model.predict(img_reshape)
        
        return prediction
if file is None:
    st.text("Please upload an image file")
else:
    image = Image.open(file)
    st.image(image, use_column_width=True)
    predictions = import_and_predict(image, model)
    score = tf.nn.softmax(predictions[0])
    st.write(predictions)
    st.write(score)
    class_names=['No_DR', 'Mild', 'Moderate', 'Severe', 'Proliferate_DR']
    print("This person most likely has {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score)))

Overwriting app.py


In [75]:
! streamlit run app.py 

^C
